In [1]:
# import pandas as pd
# from sklearn.metrics import classification_report

# # Load processed dataset
# df = pd.read_csv("../data/processed/bank_transactions_features.csv")

# # Initialize rule-based fraud column
# df['rule_fraud'] = 0

# # Define high-risk merchant categories (example)
# high_risk_merchants = ['Bitcoin', 'Cryptocurrency', 'Jewelry', 'Luxury Goods']

# # Apply rules
# df.loc[
#     (df['balance_dev'] > 50000) |  # Large deviation from customer's average balance
#     ((df['new_device'] == 1) & (df['new_location'] == 1)) |  # New device AND new location
#     (df['Merchant_Category'].isin(high_risk_merchants)),      # High-risk merchant
#     'rule_fraud'
# ] = 1

# # Classification report
# print(classification_report(df['Is_Fraud'], df['rule_fraud']))


In [2]:
# # 03_ml_modeling.ipynb

# import sys
# sys.path.append("../src")
# from modeling import split_X_y, split_train_test, scale_features, train_simple_model, evaluate_model
# import pandas as pd

# # --- Load Processed CSV ---
# df = pd.read_csv("../data/processed/bank_transactions_features.csv")

# # --- Split features / target ---
# X, y = split_X_y(df, "Is_Fraud")

# # --- Train/Test Split ---
# X_train, X_test, y_train, y_test = split_train_test(X, y, stratify=y)

# # --- Scale numeric features ---
# X_train_s, X_test_s = scale_features(X_train, X_test)

# # --- Train Logistic Regression ---
# clf = train_simple_model(X_train_s.fillna(0), y_train, problem="classification", model_name="logistic")

# # --- Evaluate ---
# metrics = evaluate_model(clf, X_test_s.fillna(0), y_test)
# print(metrics)

# # --- Save Model ---
# import joblib
# joblib.dump(clf, "../model/logistic_model.pkl")


In [3]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [4]:
df = pd.read_csv("../data/processed/bank_transactions_features.csv")
print(df.shape)


(200000, 84)


In [5]:
X = df.drop("Is_Fraud", axis=1)
y = df["Is_Fraud"]

In [7]:
joblib.dump(X.columns.tolist(), "../model/columns_used.pkl")
print("columns_used.pkl saved")


columns_used.pkl saved


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [9]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


c:\Users\Meresa\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [10]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     37982
           1       0.00      0.00      0.00      2018

    accuracy                           0.95     40000
   macro avg       0.47      0.50      0.49     40000
weighted avg       0.90      0.95      0.92     40000



c:\Users\Meresa\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Meresa\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Meresa\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
joblib.dump(model, "../model/logistic_model.pkl")
print("Model saved successfully")

Model saved successfully


In [12]:
import sys
sys.path.append("../src/")

import pandas as pd
import joblib
from modeling import split_X_y, split_train_test, scale_features, train_simple_model, evaluate_model

# --- Load processed CSV (already one-hot encoded) ---
df = pd.read_csv("../data/processed/bank_transactions_features.csv")

# --- Split X / y ---
X = df.drop(columns=["Is_Fraud"])
y = df["Is_Fraud"]

# ✅ Save column order BEFORE training
columns_used = X.columns.tolist()
joblib.dump(columns_used, "../model/columns_used.pkl")

# --- Train/Test split ---
X_train, X_test, y_train, y_test = split_train_test(X, y, stratify=y)

# --- Scale numeric features ---
X_train_s, X_test_s = scale_features(X_train, X_test)

# --- Train model ---
clf = train_simple_model(
    X_train_s.fillna(0),
    y_train,
    problem="classification",
    model_name="random_forest"   # or logistic
)

# --- Evaluate ---
metrics = evaluate_model(clf, X_test_s.fillna(0), y_test)
print(metrics)

# --- Save model ---
joblib.dump(clf, "../model/rf_model.pkl")


{'accuracy': 0.94955, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}


['../model/rf_model.pkl']